## Calling a Graph function

In [1]:
using System;
using System.IO;

string filePath = Directory.GetCurrentDirectory();
Console.WriteLine("Current file path: " + filePath);

Console.WriteLine(".NET Version: " + Environment.Version);

Current file path: c:\Users\mrochon\source\repos\ai-samples\NotebookSK\7. GraphCall
.NET Version: 8.0.1


In [7]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.MsGraph, 1.0.1-alpha"
#r "nuget: Microsoft.Graph"
#r "nuget: Azure.identity"


Installed Packages Azure.identity, 1.10.4 microsoft.graph, 4.51.0 Microsoft.SemanticKernel, 1.0.1 microsoft.semantickernel.connectors.openai, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha Microsoft.SemanticKernel.Plugins.MsGraph, 1.0.1-alpha

In [21]:
using Microsoft.Graph;
using Azure.Identity;
var scopes = new[] { "User.Read", "Mail.Send" };
var tenantId = "consumers";
// Value from app registration
var clientId = "42f18b97-5fd1-4f8c-8c6c-2c5c3dba0623";
// using Azure.Identity;
var options = new DeviceCodeCredentialOptions
{
    AuthorityHost = AzureAuthorityHosts.AzurePublicCloud,
    ClientId = clientId,
    TenantId = tenantId,
    DeviceCodeCallback = (code, cancellation) =>
    {
        Console.WriteLine(code.Message);
        return Task.FromResult(0);
    },
};
// https://learn.microsoft.com/dotnet/api/azure.identity.devicecodecredential
var deviceCodeCredential = new DeviceCodeCredential(options);
var graphClient = new GraphServiceClient(deviceCodeCredential, scopes);

In [22]:
#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Plugins.MsGraph;
using Microsoft.Graph;
using Microsoft.SemanticKernel.Plugins.MsGraph.Connectors;
var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();
kernel.Plugins.Clear();
#pragma warning disable SKEXP0053
var emailConnector = new OutlookMailConnector(graphClient);
var emailPlugin = new EmailPlugin(emailConnector);
kernel.ImportPluginFromObject(emailPlugin, "email");

List plugins and functions

In [ ]:
    foreach (KernelFunctionMetadata func in kernel.Plugins.GetFunctionsMetadata())
    {
        Console.WriteLine($"Plugin: {func.PluginName}");
        Console.WriteLine($"   {func.Name}: {func.Description}");

        if (func.Parameters.Count > 0)
        {
            Console.WriteLine("      Params:");
            foreach (var p in func.Parameters)
            {
                Console.WriteLine($"      - {p.Name}: {p.Description}");
                Console.WriteLine($"        default: '{p.DefaultValue}'");
            }
        }
        Console.WriteLine();
    }

Now try different prompts

In [24]:
#pragma warning disable SKEXP0060
    var planner = new HandlebarsPlanner();
    var goal = "Send an email to suiramor@gmail.com to say hello";
    var plan = await planner.CreatePlanAsync(kernel, goal);
    Console.WriteLine("Plan:\n");
    Console.WriteLine(plan.ToString());

Plan:

{{!-- Step 1: Set the email content --}}
{{set "content" "Hello, I hope you're having a great day!"}}

{{!-- Step 2: Set the email recipients --}}
{{set "recipients" "suiramor@gmail.com"}}

{{!-- Step 3: Set the email subject --}}
{{set "subject" "Hello from Handlebars"}}

{{!-- Step 4: Send the email --}}
{{email-SendEmail content=content recipients=recipients subject=subject}}


In [26]:
#pragma warning disable SKEXP0060
planner = new HandlebarsPlanner(System.IO.File.ReadAllText("plan.txt"));

Error: (2,33): error CS1503: Argument 1: cannot convert from 'string' to 'Microsoft.SemanticKernel.Planning.Handlebars.HandlebarsPlannerOptions?'

Execute the plan

In [10]:
    var resp = await plan.InvokeAsync(kernel);
    Console.WriteLine("Result:\n");
    Console.WriteLine(resp);

Error: (3,10): error CS1002: ; expected
(3,22): error CS1002: ; expected
(3,22): error CS1010: Newline in constant
(3,72): error CS1002: ; expected
(5,14): error CS1002: ; expected
(5,15): error CS1002: ; expected
(5,15): error CS1513: } expected
(5,27): error CS1002: ; expected
(5,41): error CS1002: ; expected
(5,48): error CS1002: ; expected
(5,57): error CS1002: ; expected
(6,18): error CS1003: Syntax error, ',' expected
(6,48): error CS1003: Syntax error, ',' expected
(6,61): error CS1002: ; expected
(7,1): error CS1010: Newline in constant
(7,3): error CS1002: ; expected
(9,52): error CS1026: ) expected
(9,53): error CS7017: Member definition, statement, or end-of-file expected